In [1]:
%%html
<script>
    // AUTORUN ALL CELLS ON NOTEBOOK-LOAD!
    require(
        ['base/js/namespace', 'jquery'], 
        function(jupyter, $) {
            $(jupyter.events).on("kernel_ready.Kernel", function () {
                console.log("Auto-running all cells-below...");
                jupyter.actions.call('jupyter-notebook:run-all-cells-below');
                jupyter.actions.call('jupyter-notebook:save-notebook');
            });
        }
    );
</script>

In [ ]:
############################################################
# pyupbit.get_current_price("KRW-BTC") : BIT 현재가
# --> price_KRW_BTC : BTC 현재가 
# upbit.get_balance(ticker="KRW-BTC") : BIT 잔고 조회 (My_Wallet_BIT)
# My_Wallet_BIT = upbit.get_balance(ticker="KRW-BTC") # <class 'float'>
# My_Wallet_Money = upbit.get_balance(ticker="KRW")
# My_Wallet_BITWON = float(price_KRW_BTC * My_Wallet_BIT))
# 
# print(upbit.get_balance("KRW-XRP"))     # KRW-XRP 조회
# print(upbit.get_balance("KRW"))         # 보유 현금 조회
# 
############################################################
import os
import jwt
import uuid
import hashlib
import pyupbit
import codecs
import datetime, time# for sleep
import pandas as pd # command 로 시작할 때는 모듈 import 위치고려
from urllib.parse import urlencode
import codecs
import shutil

import requests

from init_Stock import *

access_key = os.environ['UPBIT_OPEN_API_ACCESS_KEY']
secret_key = os.environ['UPBIT_OPEN_API_SECRET_KEY']
server_url = os.environ['UPBIT_OPEN_API_SERVER_URL']
upbit = pyupbit.Upbit(access_key, secret_key)

payload = {
    'access_key': access_key,
    'nonce': str(uuid.uuid4()),
}

jwt_token = jwt.encode(payload, secret_key) #.decode('utf-8')
authorize_token = 'Bearer {}'.format(jwt_token)
headers = {"Authorization": authorize_token}

res = requests.get(server_url + "/v1/accounts", headers=headers)

# print(res.json())
# [{'currency': 'BTC', 'balance': '0.00026', 'locked': '0.0', 'avg_buy_price': '54041000', 'avg_buy_price_modified': False, 'unit_currency': 'KRW'}]

############################################################
# print("# Dataframe")
# print(Coin_Frame)
df = pd.DataFrame(Coin_Frame) # init_Stock.py에러 호출
# print(df)

# print("###########################")
# for index, row in df.iterrows(): # init_Stock.py의 Target_Frame 값을 호출 (df.iterrows)
#     print(index, row)
#     print(row['코인명'])

############################################################
# Market 조건
Coin_Buy_Condition_List = [0]
for index, row in df.iterrows() : # init_Stock.py의 Target_Frame 값을 호출 (df.iterrows)
    if row['코인명'] != 'KRW' :
        Daily_Check_Coin = pyupbit.get_ohlcv(row['코인명'], interval="day", count=365)
        Daily_Check_Coin['diff1'] = round(((Daily_Check_Coin['high'] - Daily_Check_Coin['low']) / Daily_Check_Coin['low']) * 100,2)
        Daily_Check_Coin['diff2'] = round(((Daily_Check_Coin['open'] - Daily_Check_Coin['close']) / Daily_Check_Coin['open']) * 100,2)
        Daily_Check_Coin_Minus = Daily_Check_Coin[Daily_Check_Coin['diff2'] < 0]
        # print("# Coin 마이너스만..")
        # print(Daily_Check_Coin_Minus['diff2'].max())
        # print(Daily_Check_Coin_Minus['diff2'].min())
        # print(Daily_Check_Coin_Minus['diff2'].mean())
        Coin_Lower_Value_V = round(Daily_Check_Coin_Minus['diff2'].mean(),1)
        # print("")
        Coin_Buy_Condition_List.append(Coin_Lower_Value_V)
# print(Coin_Buy_Condition_List)
df['Buy_Condition'] = Coin_Buy_Condition_List
print(df)
print("##########################")
############################################################
# BTC 조건
Daily_Check_BTC = pyupbit.get_ohlcv("KRW-BTC", interval="day", count=365)
Daily_Check_BTC['diff1'] = round(((Daily_Check_BTC['high'] - Daily_Check_BTC['low']) / Daily_Check_BTC['low']) * 100,2)
Daily_Check_BTC['diff2'] = round(((Daily_Check_BTC['open'] - Daily_Check_BTC['close']) / Daily_Check_BTC['open']) * 100,2)
Daily_Check_BTC_Minus = Daily_Check_BTC[Daily_Check_BTC['diff2'] < 0]
print("# BTC 마이너스만..")
print(Daily_Check_BTC_Minus['diff2'].max())
print(Daily_Check_BTC_Minus['diff2'].min())
print(Daily_Check_BTC_Minus['diff2'].mean() * 2) # 엥???? 
BTC_Lower_Value_V = round(Daily_Check_BTC_Minus['diff2'].mean() * 2,1)
############################################################
# ETH 조건
Daily_Check_ETH = pyupbit.get_ohlcv("KRW-ETH", interval="day", count=365)
Daily_Check_ETH['diff1'] = round(((Daily_Check_ETH['high'] - Daily_Check_ETH['low']) / Daily_Check_ETH['low']) * 100,2)
Daily_Check_ETH['diff2'] = round(((Daily_Check_ETH['open'] - Daily_Check_ETH['close']) / Daily_Check_ETH['open']) * 100,2)
Daily_Check_ETH_Minus = Daily_Check_ETH[Daily_Check_ETH['diff2'] < 0]
print("# ETH 마이너스만..")
print(Daily_Check_ETH_Minus['diff2'].max())
print(Daily_Check_ETH_Minus['diff2'].min())
print(Daily_Check_ETH_Minus['diff2'].mean() * 2) # 엥???? 
ETH_Lower_Value_V = round(Daily_Check_ETH_Minus['diff2'].mean() * 2,1)
############################################################
# XRP 조건
Daily_Check_XRP = pyupbit.get_ohlcv("KRW-XRP", interval="day", count=365)
Daily_Check_XRP['diff1'] = round(((Daily_Check_XRP['high'] - Daily_Check_XRP['low']) / Daily_Check_XRP['low']) * 100,2)
Daily_Check_XRP['diff2'] = round(((Daily_Check_XRP['open'] - Daily_Check_XRP['close']) / Daily_Check_XRP['open']) * 100,2)
Daily_Check_XRP_Minus = Daily_Check_XRP[Daily_Check_XRP['diff2'] < 0]
print("# XRP 마이너스만..")
print(Daily_Check_XRP_Minus['diff2'].max())
print(Daily_Check_XRP_Minus['diff2'].min())
print(Daily_Check_XRP_Minus['diff2'].mean() * 2) # 엥???? 
XRP_Lower_Value_V = round(Daily_Check_XRP_Minus['diff2'].mean() * 2,1)
############################################################
BTC_Upper_Value = float(1.5) # 수익라인, UPBit 거래수수료
# BTC_Lower_Value = float(-1.5)
BTC_Lower_Value = float(BTC_Lower_Value_V)

ETH_Upper_Value = float(1.5) # 수익라인, UPBit 거래수수료
# ETH_Lower_Value = float(-1.5)
ETH_Lower_Value = float(ETH_Lower_Value_V)

XRP_Upper_Value = float(1.5) # 수익라인, UPBit 거래수수료
# XRP_Lower_Value = float(-1.5)
XRP_Lower_Value = float(XRP_Lower_Value_V)
############################################################

def Current_Coin(Market) :
    check_get_type = 0
    while check_get_type == 0 :
        time.sleep(1)
        TEMP_Coin_Current = pyupbit.get_current_price(Market) # 현재 BIT가격
        if isinstance(TEMP_Coin_Current, float) :
            check_get_type = 1
            # print("# WHILE - TEMP_Coin_Current : %s" % TEMP_Coin_Current)
            return TEMP_Coin_Current

def Current_BTC() :
    check_get_type = 0
    while check_get_type == 0 :
        time.sleep(1)
        TEMP_BTC_Current = pyupbit.get_current_price("KRW-BTC") # 현재 BIT가격
        if isinstance(TEMP_BTC_Current, float) :
            check_get_type = 1
            print("# WHILE - TEMP_BTC_Current : %s" % TEMP_BTC_Current)
            return TEMP_BTC_Current

def Current_ETH() :
    check_get_type = 0
    while check_get_type == 0 :
        time.sleep(1)
        TEMP_ETH_Current = pyupbit.get_current_price("KRW-ETH") # 현재 BIT가격
        if isinstance(TEMP_ETH_Current, float) :
            check_get_type = 1
            print("# WHILE - TEMP_ETH_Current : %s" % TEMP_ETH_Current)
            return TEMP_ETH_Current

def Current_XRP() :
    check_get_type = 0
    while check_get_type == 0 :
        time.sleep(1)
        TEMP_XRP_Current = pyupbit.get_current_price("KRW-XRP") # 현재 BIT가격
        if isinstance(TEMP_XRP_Current, float) :
            check_get_type = 1
            print("# WHILE - TEMP_XRP_Current : %s" % TEMP_XRP_Current)
            return TEMP_XRP_Current

# price_KRW_BTC = pyupbit.get_current_price("KRW-BTC") # BIT 현재가
# price_KRW_ETH = pyupbit.get_current_price("KRW-ETH") # BIT 현재가
price_KRW_BTC = Current_BTC()
price_KRW_ETH = Current_ETH()
price_KRW_XRP = Current_XRP()

Min_Call_Price = float(10000)
Min_Call_BTC_Volume = Min_Call_Price / price_KRW_BTC # 10000원을 현재 BIT가로 나눈값
Min_Call_ETH_Volume = Min_Call_Price / price_KRW_ETH # 10000원을 현재 ETH가로 나눈값
Min_Call_XRP_Volume = Min_Call_Price / price_KRW_XRP # 10000원을 현재 XRP가로 나눈값

def Call_Condition(File_Name) :

    output = codecs.open(File_Name, "a", "utf-8-sig") 

    payload = {
        'access_key': access_key,
        'nonce': str(uuid.uuid4()),
    }

    jwt_token = jwt.encode(payload, secret_key) #.decode('utf-8')
    authorize_token = 'Bearer {}'.format(jwt_token)
    headers = {"Authorization": authorize_token}

    res = requests.get(server_url + "/v1/accounts", headers=headers)

    # 자산 종류 수로 내가 가진 자산을 확인 2이면 BIT와 KRW, 1개면 KRW로 판단 (BIT없음)
    # ---> 1개일 경우 종류를 확인해봐야할 듯... (2021.03.30)

    My_Wallet_KRW, My_Wallet_BTC, My_Wallet_ETH = {}, {}, {}
    if len(res.json()) >= 2 :
        for x in res.json() :
            if x["currency"] == "KRW" :
                My_Wallet_KRW = x
            elif x["currency"] == "BTC" :
                My_Wallet_BTC = x
                if len(My_Wallet_BTC) == 0 :
                    print("# %s : My_Wallet_BTC == 0 in Call_Condition" % datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"))
                    # __Make_Put("BUY", "BTC", 10000, 10000, OneDay_BUY_Count, File_Name) # 조건, 최소금액, 최소BIT, File_Name
            elif x["currency"] == "ETH" :
                My_Wallet_ETH = x
                if len(My_Wallet_ETH) == 0 :
                    print("# %s : My_Wallet_BTC == 0 in Call_Condition" % datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"))
                    # __Make_Put("BUY", "ETH", 10000, 10000, OneDay_BUY_Count, File_Name) # 조건, 최소금액, 최소ETH, File_Name
            elif x["currency"] == "XRP" :
                My_Wallet_XRP = x
                if len(My_Wallet_XRP) == 0 :
                    print("# %s : My_Wallet_BTC == 0 in Call_Condition" % datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"))
                    # __Make_Put("BUY", "XRP", 10000, 10000, OneDay_BUY_Count, File_Name) # 조건, 최소금액, 최소ETH, File_Name
            else :
                print("# ELSE")
    else :
        print("# res.json LENGTH is Wrong")

    ############################################################
    # Upper_Value = float(1.5) # 수익라인, UPBit 거래수수료
    # Lower_Value = float(-15) # 물타기라인, UPBit 거래수수료
    # Min_Call_Price = float(10000)
    # price_KRW_BTC = pyupbit.get_current_price("KRW-BTC") # BIT 현재가
    # Min_Call_BTC_Volume = 10000 / price_KRW_BTC
    print("# 최소매수가능 금액 : %f" % Min_Call_Price)
    print("# 최소매수가능 BIT : %f" % Min_Call_BTC_Volume)
    print("# 최소매수가능 ETH : %f" % Min_Call_ETH_Volume)
    print("# 최소매수가능 XRP : %f" % Min_Call_XRP_Volume)
    ############################################################

    ############################################################
    Call_Price_Possible = float(My_Wallet_KRW['balance']) / 10 # 보유 금액의 1/10
    Call_Volume_Possible = upbit.get_balance(ticker="KRW-BTC") / 10 # 보유 BIT의 1/10
    print("# 1/10 Volume = %f" % Call_Volume_Possible)
    output.write("# 1/10 Volume = %f\n" % Call_Volume_Possible)
    print("# 1/10 Price = %f" % Call_Price_Possible)
    output.write("# 1/10 Price = %f\n" % Call_Price_Possible)
    print("")
    output.write("\n")
    ############################################################

    if Call_Price_Possible < Min_Call_Price : # 가진 자산의 1/10을 매도하거나 매수하지만 기본금액에 모자라면 기본금액으로 설정
        i = 1
        Temp_Possible = 0
        print("Min_Call_Price : %s" % Min_Call_Price) # <class 'float'>
        print("Call_Price_Possbile : %s" % Call_Price_Possible) # <class 'float'>
        while Call_Price_Possible < Min_Call_Price : # 현재가 변동으로 10%를 추가로 잡아줌
            if Temp_Possible == 0 : 
                Temp_Possible = Call_Price_Possible
            Call_Price_Possible = Temp_Possible * i
            print("# %s번째 Price : %s" % (i, Call_Price_Possible))
            output.write("# %s번째 Price : %s\n" % (i, Call_Price_Possible))
            i += 1
            if i > 10 :
                Call_Price_Possible = Min_Call_Price

        print("# 완료 %s번째 Price : %s" % (i, Call_Price_Possible))
        output.write("# 완료 %s번째 Price : %s\n" % (i, Call_Price_Possible))
        print("")
        output.write("\n")

    # Temp_Volume = Call_Volume_Possible * price_KRW_BTC
    if Call_Volume_Possible < Min_Call_BTC_Volume : # 가진 자산의 1/10을 매도하거나 매수하지만 기본금액에 모자라면 기본금액으로 설정
        i = 1
        while (Call_Volume_Possible * price_KRW_BTC) < Min_Call_Price :
            print("# %s번째 Volume : %f" % (i, Call_Volume_Possible))
            output.write("# %s번째 Volume : %f\n" % (i, Call_Volume_Possible))
            Call_Volume_Possible = Call_Volume_Possible * i
            # if i < 0 :
            #     Call_Volume_Possible = price_KRW_BTC / Min_Call_Price
            #     break
            i += 1
        print("# 완료 %s번째 Volume : %f" % (i, Call_Volume_Possible))
        output.write("# 완료 %s번째 Volume : %f\n" % (i, Call_Volume_Possible))
        print("")
        output.write("\n")

    print("# 최종가격대")
    output.write("# 최종가격대\n")
    print("# Call_Price_Possible = %f" % Call_Price_Possible)
    output.write("# Call_Price_Possible = %f\n" % Call_Price_Possible)
    print("# Call_Volume_Possible = %f" % Call_Volume_Possible)
    output.write("# Call_Volume_Possible = %f\n" % Call_Volume_Possible)
    print("")
    output.write("\n")

    output.close()

    return Call_Price_Possible, Call_Volume_Possible

def __Make_OrderLists(COIN) :

    # print("# Market : %s" % COIN)

    Total_Order = []
    Trade_List_uuid, Trade_List_side, Trade_List_date = [], [], []
    if COIN == 'KRW-BTC' :
        # print("# KRW-BTC 주문리스트 조회")
        for Dist in ('done','cancel') :
            query = {
            'market': 'KRW-BTC',
            'state': Dist, # 주문 상태 , wait : 체결 대기 (default),  done : 전체 체결 완료,  cancel : 주문 취소
            }
            query_string = urlencode(query).encode()
            m = hashlib.sha512()
            m.update(query_string)
            query_hash = m.hexdigest()
            payload = {
                'access_key': access_key,
                'nonce': str(uuid.uuid4()),
                'query_hash': query_hash,
                'query_hash_alg': 'SHA512',
            }
            jwt_token = jwt.encode(payload, secret_key)
            authorize_token = 'Bearer {}'.format(jwt_token)
            headers = {"Authorization": authorize_token}
            res = requests.get(server_url + "/v1/orders", params=query, headers=headers)
            for res_list in range(len(res.json())) :
                Trade_List_uuid.append(res.json()[res_list]["uuid"])
                Trade_List_side.append(res.json()[res_list]["side"])
                Trade_List_date.append(res.json()[res_list]["created_at"])
    elif COIN == 'KRW-ETH' :
        # print("# KRW-ETH 주문리스트 조회")
        for Dist in ('done','cancel') :
            query = {
            'market': 'KRW-ETH',
            'state': Dist, # 주문 상태 , wait : 체결 대기 (default),  done : 전체 체결 완료,  cancel : 주문 취소
            }
            query_string = urlencode(query).encode()
            m = hashlib.sha512()
            m.update(query_string)
            query_hash = m.hexdigest()
            payload = {
                'access_key': access_key,
                'nonce': str(uuid.uuid4()),
                'query_hash': query_hash,
                'query_hash_alg': 'SHA512',
            }
            jwt_token = jwt.encode(payload, secret_key)
            authorize_token = 'Bearer {}'.format(jwt_token)
            headers = {"Authorization": authorize_token}
            res = requests.get(server_url + "/v1/orders", params=query, headers=headers)
            for res_list in range(len(res.json())) :
                Trade_List_uuid.append(res.json()[res_list]["uuid"])
                Trade_List_side.append(res.json()[res_list]["side"])
                Trade_List_date.append(res.json()[res_list]["created_at"])
    elif COIN == 'KRW-XRP' :
        # print("# KRW-XRP 주문리스트 조회")
        for Dist in ('done','cancel') :
            query = {
            'market': 'KRW-XRP',
            'state': Dist, # 주문 상태 , wait : 체결 대기 (default),  done : 전체 체결 완료,  cancel : 주문 취소
            }
            query_string = urlencode(query).encode()
            m = hashlib.sha512()
            m.update(query_string)
            query_hash = m.hexdigest()
            payload = {
                'access_key': access_key,
                'nonce': str(uuid.uuid4()),
                'query_hash': query_hash,
                'query_hash_alg': 'SHA512',
            }
            jwt_token = jwt.encode(payload, secret_key)
            authorize_token = 'Bearer {}'.format(jwt_token)
            headers = {"Authorization": authorize_token}
            res = requests.get(server_url + "/v1/orders", params=query, headers=headers)
            for res_list in range(len(res.json())) :
                Trade_List_uuid.append(res.json()[res_list]["uuid"])
                Trade_List_side.append(res.json()[res_list]["side"])
                Trade_List_date.append(res.json()[res_list]["created_at"])
    else : # bid : 매수, ask : 매도
        print("# BTC/ETH 주문내역 조회 : 오류")
        # ---> 매수는 cancel 로 잡히고 매도는 done으로 잡힌다
        # ---> 매수 시 시장가로 매수하니 소수점이하..어쩌고 잔량이 어쩌고..

    Now_DATE = datetime.datetime.today().strftime("%Y-%m-%d")

    # List에 아무것도 안담기는 경우는???
    # print(Trade_List_uuid) # <class 'list'>
    # print(Trade_List_side) # <class 'list'>
    # print(Trade_List_date) # <class 'list'>
    
    Trade_List_date_10, Trade_List_side_10, idx_list = [], [], []
    for x in Trade_List_date :
        Trade_List_date_10.append(x[:10])
    # print(Trade_List_date_10)
    for idx, val in enumerate(Trade_List_date_10) :
        if val == Now_DATE :
           idx_list.append(idx) 
    # print(idx_list)
    for idx, val in enumerate(Trade_List_side) :
        if idx in idx_list :
            # print("%s : %s" % (Trade_List_date_10[idx], val))
            Trade_List_side_10.append(val)
    # print("# 길이 : %s" % len(Trade_List_uuid))
    # print("# 길이 : %s" % len(Trade_List_side))
    # print("# 길이 : %s" % len(Trade_List_date))

    print("# BID Count : %s" % Trade_List_side.count('bid')) # bid : 매수, ask : 매도
    print("# ASK Count : %s" % Trade_List_side.count('ask')) # bid : 매수, ask : 매도
    print("# Real BID Count #")
    print("# BID Count : %s" % Trade_List_side_10.count('bid')) # bid : 매수, ask : 매도

    return Trade_List_side_10.count('bid') # 매수 건수 만 Return

        # {'uuid': '418f0313-e001-4cc3-9dcc-f656dd24f3b2', uuid	주문의 고유 아이디	String
        #  'side': 'ask', side	주문 종류	String
        #  'ord_type': 'market', ord_type	주문 방식	String
        #  'price': None, price	주문 당시 화폐 가격	NumberString
        #  'state': 'done', state	주문 상태	String
        #  'market': 'KRW-BTC', market	마켓의 유일키	String
        #  'created_at': '2021-02-20T10:40:42+09:00', created_at	주문 생성 시간	DateString
        #  'volume': '0.000078', volume	사용자가 입력한 주문 양	NumberString
        #  'remaining_volume': '0.0', remaining_volume	체결 후 남은 주문 양	NumberString
        #  'reserved_fee': '0.0', reserved_fee	수수료로 예약된 비용	NumberString
        #  'remaining_fee': '0.0', remaining_fee	남은 수수료	NumberString
        #  'paid_fee': '2.509884', paid_fee	사용된 수수료	NumberString
        #  'locked': '0.0', locked	거래에 사용중인 비용	NumberString
        #  'executed_volume': '0.000078', executed_volume	체결된 양	NumberString
        #  'trades_count': 1} trade_count	해당 주문에 걸린 체결 수	Integer

def Detail_Trade(Trade_List_uuid) :

    print("# Detail_Trace Funtion START")

    for UUID in Trade_List_uuid :
        query = {
            'uuid': UUID,
        }
        query_string = urlencode(query).encode()

        m = hashlib.sha512()
        m.update(query_string)
        query_hash = m.hexdigest()

        payload = {
            'access_key': access_key,
            'nonce': str(uuid.uuid4()),
            'query_hash': query_hash,
            'query_hash_alg': 'SHA512',
        }

        jwt_token = jwt.encode(payload, secret_key)
        authorize_token = 'Bearer {}'.format(jwt_token)
        headers = {"Authorization": authorize_token}

        res = requests.get(server_url + "/v1/order", params=query, headers=headers)

        # print(res.json())
        Detail_Trade_Value = res.json()
        print(Detail_Trade_Value['side'])

    print("# Detail_Trace Funtion DONE")

def __Make_Put(CALL, COIN, Call_Possible, Min_Call_Price, OneDay_BUY_Count, File_Name) :
    # CALL : 조건(BUY/SELL), COIN : 코인명, Call_Possible : 보유코인수, Min_Call_Price : 최소매수금액, 매수건수, 파일명
    # My_Coin = upbit.get_balance(ticker=x['currency']) # <class 'float'>
    # Min_Call_Price = df.loc[df['코인'].str.match(str(x['currency'])),'1회매수액'].to_string()

    # volume : 주문량 (지정가, 시장가 매도 시 필수)
    # price : 주문 가격. (지정가, 시장가 매수 시 필수)
            # ex) KRW-BTC 마켓에서 1BTC당 1,000 KRW로 거래할 경우, 값은 1000 이 된다.
            # ex) KRW-BTC 마켓에서 1BTC당 매도 1호가가 500 KRW 인 경우, 시장가 매수 시 값을 1000으로 세팅하면 2BTC가 매수된다.
            # (수수료가 존재하거나 매도 1호가의 수량에 따라 상이할 수 있음)

    # Call_Possible 값은 <--- My_Wallet_BIT = upbit.get_balance(ticker="KRW-BTC") # 자산 내 BIT
    # Min_Call_Price = float(10000)

    # output = open(File_Name, "a")
    output = codecs.open(File_Name, "a", "utf-8-sig") 
    print("# %s : __Make_Put Function Start" % datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"))
    output.write("# %s : __Make_Put Function Start\n" % datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"))

    print("# %s : %s CALL Volume/Price : %f개 or 원" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), COIN, Call_Possible))
    output.write("# %s : %s CALL Volume/Price : %f개 or 원\n" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), COIN, Call_Possible))
    # 시장가 주문은 ord_type 필드를 price or market 으로 설정해야됩니다.
    if CALL == "SELL" :
        # 매도 주문의 경우 ord_type을 market로 설정하고 price을 null 혹은 제외해야됩니다.
        # Call_Possible = upbit.get_balance(ticker="KRW-BTC") # 매도할 때는 전체를 매도하니간 굳이 계산없이 전체
        if COIN == "BTC" :
            print("# %s : BIT 잔고 조회 : %f" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), upbit.get_balance(ticker="KRW-BTC"))) # 특정 코인이나 원화의 잔고만 조회
            output.write("# %s : BIT 잔고 조회 : %f\n" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), upbit.get_balance(ticker="KRW-BTC"))) # 특정 코인이나 원화의 잔고만 조회
            price_KRW_BTC = Current_BTC()
            # price_KRW_BTC = pyupbit.get_current_price("KRW-BTC") # <class 'float'> ---> BIT가격은 계속 바뀔테니 다시 조회
            # price_KRW_ETH = pyupbit.get_current_price("KRW-ETH") # <class 'float'> ---> BIT가격은 계속 바뀔테니 다시 조회
            # price_KRW_XRP = pyupbit.get_current_price("KRW-XRP") # <class 'float'> ---> XRP가격은 계속 바뀔테니 다시 조회
            TEMP_Call_Possible = Call_Possible * price_KRW_BTC # 현재 보유 BIT개수 * 현재 BIT값
        elif COIN == "ETH" :
            print("# %s : ETH 잔고 조회 : %f" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), upbit.get_balance(ticker="KRW-ETH"))) # 특정 코인이나 원화의 잔고만 조회
            output.write("# %s : ETH 잔고 조회 : %f\n" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), upbit.get_balance(ticker="KRW-ETH"))) # 특정 코인이나 원화의 잔고만 조회
            price_KRW_ETH = Current_ETH()
            TEMP_Call_Possible = Call_Possible * price_KRW_ETH # 현재 보유 BIT개수 * 현재 BIT값
        elif COIN == "XRP" :
            print("# %s : XRP 잔고 조회 : %f" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), upbit.get_balance(ticker="KRW-XRP"))) # 특정 코인이나 원화의 잔고만 조회
            output.write("# %s : XRP 잔고 조회 : %f\n" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), upbit.get_balance(ticker="KRW-XRP"))) # 특정 코인이나 원화의 잔고만 조회
            price_KRW_XRP = Current_XRP()
            TEMP_Call_Possible = Call_Possible * price_KRW_XRP # 현재 보유 XRP개수 * 현재 XRP값
        else :
            print("# COIN Error")

        if TEMP_Call_Possible < float(5000) : # 최소금액에 못미치면 그냥 returne
            print("# %s :---> 매도 COIN이 최소금액이 안됨" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S")))
            output.write("# %s :---> 매도 COIN이 최소금액이 안됨\n" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S")))
            return None
        else :
            print("# %s : Make call SELL -> BTC" % datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"))
            output.write("# %s : Make call SELL -> BTC\n" % datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"))
            CALL = "ask" # 매도
            ORD_TYPE = "market"
            PRICE = ''
            VOLUME = Call_Possible

    # CALL : 조건(BUY/SELL), COIN : 코인명, Call_Possible : 보유코인수, Min_Call_Price : 최소매수금액, 매수건수, 파일명
    # My_Coin = upbit.get_balance(ticker=x['currency']) # <class 'float'>
    # Min_Call_Price = df.loc[df['코인'].str.match(str(x['currency'])),'1회매수액'].to_string()
    elif CALL == "BUY" :
        OneDay_BUY_Count += 1
        # if COIN == "BTC" :
        #     TEMP_Call_Possible = Call_Possible * price_KRW_BTC # 현재 보유 COIN개수 * 현재 BIT값
        # elif COIN == "ETH" :
        #     TEMP_Call_Possible = Call_Possible * price_KRW_ETH # 현재 보유 COIN개수 * 현재 ETH값
        # elif COIN == "XRP" :
        #     TEMP_Call_Possible = Call_Possible * price_KRW_XRP # 현재 보유 COIN개수 * 현재 XRP값
        # else :
        #     print("# %s : COIN 오류" % datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"))
        #     output.write("# %s : COIN 오류\n" % datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"))

        My_WON = upbit.get_balance("KRW")         # 보유 현금 조회
        if OneDay_BUY_Count > 3 :
            return "3"
        # 매수 주문의 경우 ord_type을 price로 설정하고 volume을 null 혹은 제외해야됩니다.
        if Min_Call_Price < float(5000) : # 최소금액에 못미치면 그냥 return
        # CALL : 조건(BUY/SELL), COIN : 코인명, Call_Possible : 보유코인수, Min_Call_Price : 최소매수금액, 매수건수, 파일명
            print("# %s :---> 매수해야되지만 최소금액이 안됨" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S")))
            output.write("# %s :---> 매수해야되지만 최소금액이 안됨\n" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S")))
            return None
        elif My_WON < float(5000) :
            print("# %s :---> 매수해야되지만 보유금액이 안됨" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S")))
            output.write("# %s :---> 매수해야되지만 보유금액이 안됨\n" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S")))
            return None
        else :
            print("# %s : Make call BUY" % datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"))
            output.write("# %s : Make call BUY\n" % datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"))
            CALL = "bid" # 매수
            ORD_TYPE = "price"
            VOLUME = ''
            PRICE = Min_Call_Price
    else :
        print("# %s : Make Call ERROR~!" % datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"))
        output.write("# %s : Make Call ERROR~!\n" % datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"))

    if COIN == "BTC" :
        query = {
        'market': 'KRW-BTC',
        'side': CALL, # 주문 종류 (필수) - bid : 매수, ask : 매도
        'volume': VOLUME, # 주문 수량, 주문량 (지정가, 시장가 매도 시 필수) NumberString
        'price': PRICE, # 유닛당 주문 가격
        'ord_type': ORD_TYPE,
        }
    elif COIN == "ETH" :
        query = {
        'market': 'KRW-ETH',
        'side': CALL, # 주문 종류 (필수) - bid : 매수, ask : 매도
        'volume': VOLUME, # 주문 수량, 주문량 (지정가, 시장가 매도 시 필수) NumberString
        'price': PRICE, # 유닛당 주문 가격
        'ord_type': ORD_TYPE,
        }
    elif COIN == "XRP" :
        query = {
        'market': 'KRW-XRP',
        'side': CALL, # 주문 종류 (필수) - bid : 매수, ask : 매도
        'volume': VOLUME, # 주문 수량, 주문량 (지정가, 시장가 매도 시 필수) NumberString
        'price': PRICE, # 유닛당 주문 가격
        'ord_type': ORD_TYPE,
        }
    else :
        print("# 주문오류~~~")
        output.write("# 주문오류~~~\n")
    
    print(query)

        # 주문 타입 (필수) - limit : 지정가 주문, price : 시장가 주문(매수), market : 시장가 주문(매도)
        # identifier : 조회용 사용자 지정값 (선택) String (Uniq 값 사용)
    query_string = urlencode(query).encode()

    print("# Query : ") # {'market': 'KRW-BTC', 'side': 'bid', 'volume': '', 'price': 6689.808521999999, 'ord_type': 'price'}
    # print(query)
    # print(type(query)) # <class 'dict'>
    output.write("# Query : ") # {'market': 'KRW-BTC', 'side': 'bid', 'volume': '', 'price': 6689.808521999999, 'ord_type': 'price'}
    output.write(str(query))

    m = hashlib.sha512()
    m.update(query_string)
    query_hash = m.hexdigest()

    payload = {
        'access_key': access_key,
        'nonce': str(uuid.uuid4()),
        'query_hash': query_hash,
        'query_hash_alg': 'SHA512',
    }

    jwt_token = jwt.encode(payload, secret_key) # .decode('utf-8')
    authorize_token = 'Bearer {}'.format(jwt_token)
    headers = {"Authorization": authorize_token}

    # print(jwt_token)
    # print(authorize_token)
    # print(headers)
    res = requests.post(server_url + "/v1/orders", params=query, headers=headers)

    # print("# %s : __Make_Put : Complete\n" % datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"))
    # output.write("# %s : __Make_Put : Complete\n" % datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"))
    # print(type(res.text)) # <class 'str'>

    # print(res.text) # "uuid":"ac1def11-d5f2-45e9-9d57-5fc7b998d92d","side":"bid","ord_type":"price","price":"10000.0","state":"wait","market":"KRW-BTC","created_at":"2021-04-16T23:45:50+09:00","volume":null,"remaining_volume":null,"reserved_fee":"5.0","remaining_fee":"5.0","paid_fee":"0.0","locked":"10005.0","executed_volume":"0.0","trades_count":0}

     # 매수 : {"error":{"message":"주문가능한 금액(BTC)이 부족합니다.","name":"insufficient_funds_ask"}}
     # 매수 : {"uuid":"9ddd3561-1188-4e5a-83fb-5810bb7f0c94","side":"bid","ord_type":"price","price":"6689.8","state":"wait","market":"KRW-BTC","created_at":"2021-02-27T09:00:29+09:00","volume":null,"remaining_volume":null,"reserved_fee":"3.3449","remaining_fee":"3.3449","paid_fee":"0.0","locked":"6693.1449","executed_volume":"0.0","trades_count":0}
    output_res_text = res.text
    output.write("# %s : %s \n" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), output_res_text))
    output.write("\n")

    print("# %s : __Make_Put Function Done" % datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"))
    output.write("# %s : __Make_Put Function Done\n" % datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"))

    time.sleep(3) # ---> 매도하고나서 바로 또 매도하려고 함. (매도했는데 매도하려는 물량이 없어서 최소수량보다 작다는 메세지 출력됨)

    output.close()

def Current_Query(OneDay_BUY_Count, File_Name) :

    print("# Current_Query Function Start")

    payload = {
        'access_key': access_key,
        'nonce': str(uuid.uuid4()),
    }

    jwt_token = jwt.encode(payload, secret_key) #.decode('utf-8')
    authorize_token = 'Bearer {}'.format(jwt_token)
    headers = {"Authorization": authorize_token}

    res = requests.get(server_url + "/v1/accounts", headers=headers)

    print(res.json())
    print(type(res.json()))

    My_Wallet_KRW, My_Wallet_BTC, My_Wallet_ETH, My_Wallet_XRP = {}, {}, {}, {}
    if len(res.json()) >= 2 :
        for x in res.json() :
            if x["currency"] == "KRW" :
                My_Wallet_KRW = x
            elif x["currency"] == "BTC" :
                My_Wallet_BTC = x
            elif x["currency"] == "ETH" :
                My_Wallet_ETH = x
            elif x["currency"] == "XRP" :
                My_Wallet_XRP = x
            else :
                print("# ELSE")

        if len(My_Wallet_BTC) == 0 :
            __Make_Put("BUY", "BTC", 10000, 10000, OneDay_BUY_Count, File_Name) # 조건, 최소금액, 최소BIT, File_Name
            time.sleep(1)
        if len(My_Wallet_ETH) == 0 :
            __Make_Put("BUY", "ETH", 10000, 10000, OneDay_BUY_Count, File_Name) # 조건, 최소금액, 최소ETH, File_Name
            time.sleep(1)
        if len(My_Wallet_XRP) == 0 :
            __Make_Put("BUY", "XRP", 10000, 10000, OneDay_BUY_Count, File_Name) # 조건, 최소금액, 최소ETH, File_Name
            time.sleep(1)
    else :
        print("# res.json LENGTH is 1 이하")
        output.write("# res.json LENGTH is 1 이하")

    if len(My_Wallet_BTC) != 0 :
        print("# BTC : %s" % My_Wallet_BTC["balance"]) # <class 'str'>
    if len(My_Wallet_ETH) != 0 :
        print("# ETH : %s" % My_Wallet_ETH["balance"]) # <class 'str'>
    if len(My_Wallet_XRP) != 0 :
        print("# XRP : %s" % My_Wallet_XRP["balance"]) # <class 'str'>

    # if not isinstance(My_Wallet_BTC, float) :
    if isinstance(My_Wallet_BTC, dict) :
        if len(My_Wallet_BTC) == 6 :
            TEMP_BTC_My_Wallet = float(My_Wallet_BTC['avg_buy_price'])
            # My_Wallet_BTC = upbit.get_balance(ticker="KRW-BTC") # <class 'float'>
            # TEMP_BTC_Current = pyupbit.get_current_price("KRW-BTC")
            TEMP_BTC_Current = Current_BTC()
            if int(TEMP_BTC_My_Wallet) <= 0 :
                print("# TEMP My Wallet : %f ---> 보유 BTC가 없음" % TEMP_BTC_My_Wallet)
                output.write("# TEMP My Wallet : %f ---> 보유 BTC가 없음\n" % TEMP_BTC_My_Wallet)
                return TEMP_BTC_My_Wallet
            TEMP_BTC_Percent = (TEMP_BTC_Current - TEMP_BTC_My_Wallet) / TEMP_BTC_My_Wallet * 100
            TEMP_BTC_Percent = round(TEMP_BTC_Percent, 2)
            print("# 현재 수익률 : %f " % TEMP_BTC_Percent) # <class 'float'> 
        else :
            # __Make_Put("BUY", "BTC", 10000, 10000, OneDay_BUY_Count, File_Name)
            time.sleep(3) # 매수하고 바로 계좌조회는 에러가 나는거 같아서 추가 ---> TypeError: must be real number, not dict

    # if not isinstance(My_Wallet_ETH, float) :
    if isinstance(My_Wallet_ETH, dict) :
        if len(My_Wallet_ETH) == 6 :
            TEMP_ETH_My_Wallet = float(My_Wallet_ETH['avg_buy_price'])
            # My_Wallet_ETH = upbit.get_balance(ticker="KRW-ETH") # <class 'float'>
            TEMP_ETH_Current = Current_ETH()
            # TEMP_ETH_Current = pyupbit.get_current_price("KRW-ETH")
            if int(TEMP_ETH_My_Wallet) <= 0 :
                print("# TEMP My Wallet : %f ---> 보유 ETH가 없음" % TEMP_ETH_My_Wallet)
                output.write("# TEMP My Wallet : %f ---> 보유 ETH가 없음\n" % TEMP_ETH_My_Wallet)
                return TEMP_ETH_My_Wallet
            TEMP_ETH_Percent = (TEMP_ETH_Current - TEMP_ETH_My_Wallet) / TEMP_ETH_My_Wallet * 100
            TEMP_ETH_Percent = round(TEMP_ETH_Percent, 2)
            print("# 현재 수익률 : %f " % TEMP_ETH_Percent) # <class 'float'> 
        else :
            # __Make_Put("BUY", "ETH", 10000, 10000, OneDay_BUY_Count, File_Name)
            time.sleep(3) # 매수하고 바로 계좌조회는 에러가 나는거 같아서 추가 ---> TypeError: must be real number, not dict

    # if not isinstance(My_Wallet_XRP, float) :
    if isinstance(My_Wallet_XRP, dict) :
        if len(My_Wallet_XRP) == 6 :
            TEMP_XRP_My_Wallet = float(My_Wallet_XRP['avg_buy_price'])
            # My_Wallet_XRP = upbit.get_balance(ticker="KRW-XRP") # <class 'float'>
            # TEMP_XRP_Current = pyupbit.get_current_price("KRW-XRP")
            TEMP_XRP_Current = Current_XRP()
            if int(TEMP_XRP_My_Wallet) <= 0 :
                print("# TEMP My Wallet : %f ---> 보유 XRP가 없음" % TEMP_XRP_My_Wallet)
                output.write("# TEMP My Wallet : %f ---> 보유 XRP가 없음\n" % TEMP_XRP_My_Wallet)
                return TEMP_XRP_My_Wallet
            TEMP_XRP_Percent = (TEMP_XRP_Current - TEMP_XRP_My_Wallet) / TEMP_XRP_My_Wallet * 100
            TEMP_XRP_Percent = round(TEMP_XRP_Percent, 2)
            print("# 현재 수익률 : %f " % TEMP_XRP_Percent) # <class 'float'> 
        else :
            # __Make_Put("BUY", "XRP", 10000, 10000, OneDay_BUY_Count, File_Name)
            time.sleep(3) # 매수하고 바로 계좌조회는 에러가 나는거 같아서 추가 ---> TypeError: must be real number, not dict

    print("############################################################")
    output.write("############################################################\n")

    if len(My_Wallet_BTC) != 0 :
        print("# BTC 현재가 : %s 원" % price_KRW_BTC)
        output.write("# KRW-BTC 현재가 : %s 원\n" % price_KRW_BTC)
        print("# BIT 잔고 조회 : %s" % My_Wallet_BTC["balance"]) # 특정 코인이나 원화의 잔고만 조회
        output.write("# KRW-BTC 현재가 : %s 원\n" % price_KRW_BTC)
        # My_Wallet_BIT = upbit.get_balance(ticker="KRW-BTC") # <class 'float'>
        BIT_WON = float(My_Wallet_BTC["balance"]) * price_KRW_BTC
        print("# 현재 BIT원화 잔고 : %f 원, 수익율 : %f" % (BIT_WON, TEMP_BTC_Percent))
        output.write("# 현재 BIT원화 잔고 : %f 원, 수익율 : %f" % (BIT_WON, TEMP_BTC_Percent))
    
    if len(My_Wallet_ETH) != 0 :
        print("# ETH 현재가 : %s 원" % price_KRW_ETH)
        output.write("# KRW-ETH 현재가 : %s 원\n" % price_KRW_ETH)
        print("# ETH 잔고 조회 : %s" % My_Wallet_ETH["balance"]) # 특정 코인이나 원화의 잔고만 조회 # TypeError: must be real number, not dict
        output.write("# KRW-ETH 현재가 : %s 원\n" % price_KRW_ETH)
        # My_Wallet_ETH = upbit.get_balance(ticker="KRW-ETH") # <class 'float'>
        ETH_WON = float(My_Wallet_ETH["balance"]) * price_KRW_ETH
        print("# 현재 ETH원화 잔고 : %f 원, 수익율 : %f" % (ETH_WON, TEMP_ETH_Percent))

    if len(My_Wallet_XRP) != 0 :
        print("# XRP 현재가 : %s 원" % price_KRW_XRP)
        output.write("# KRW-XRP 현재가 : %s 원\n" % price_KRW_XRP)
        print("# XRP 잔고 조회 : %s" % My_Wallet_XRP["balance"]) # 특정 코인이나 원화의 잔고만 조회 # TypeError: must be real number, not dict
        output.write("# KRW-XRP 현재가 : %s 원\n" % price_KRW_XRP)
        # My_Wallet_XRP = upbit.get_balance(ticker="KRW-XRP") # <class 'float'>
        XRP_WON = float(My_Wallet_XRP["balance"]) * price_KRW_XRP
        print("# 현재 XRP원화 잔고 : %f 원, 수익율 : %f" % (XRP_WON, TEMP_XRP_Percent))

    if len(My_Wallet_KRW) != 0 :
        My_Wallet_KRW = My_Wallet_KRW["balance"]
        print("# 현재 원화 잔고 조회 : %s" % My_Wallet_KRW) # 특정 코인이나 원화의 잔고만 조회
        output.write("# 현재 원화 잔고 조회 : %s\n" % My_Wallet_KRW) # 특정 코인이나 원화의 잔고만 조회
        print("############################################################")
        output.write("############################################################\n")

    return len(res.json())

def Start_BIT_Trade(Call_Price_Possible, Call_Volume_Possible, File_Name) :

    OneDay_BUY_Count = 0 # 하루에 매수하는 회수를 제어하기 위한 Count, PUT(BUY) 호출할 때마다 1 Count (Start_BIT_Trade에서 최초 0부터 해서 시작)
    infinite_while = 1

    Check_Hour = []
    while infinite_while == 1 :
        # output = open(File_Name, "a")
        output = codecs.open(File_Name, "a", "utf-8-sig") 

        print("# %s : WHILE Start " % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S")))
        output.write("# %s : WHILE Start\n" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S")))
        now_time = datetime.datetime.today().strftime("%Y%m%d %H:%M:%S")

        ############################################################
        # Loop 안에서 새롭게 조회를 해야 한다
        payload = {
            'access_key': access_key,
            'nonce': str(uuid.uuid4()),
        }

        jwt_token = jwt.encode(payload, secret_key) #.decode('utf-8')
        authorize_token = 'Bearer {}'.format(jwt_token)
        headers = {"Authorization": authorize_token}

        res = requests.get(server_url + "/v1/accounts", headers=headers)

        My_Wallet_KRW, My_Wallet_BTC, My_Wallet_ETH, My_Wallet_XRP = {}, {}, {}, {}

        if len(res.json()) < 2 : # 보유자산 1개 뿐임 
            print("# 보유자산 1개 뿐임 (BIT or WON)")
            Temp_Check = res.json()[0]
            if Temp_Check['currency'] == 'KRW' :
                print("# 보유자산 1개 = WON만")
                continue
            elif Temp_Check['currency'] == 'BTC' :
                print("# 보유자산 1개 = BIT만)")
                continue
            elif Temp_Check['currency'] == 'ETH' :
                print("# 보유자산 1개 = ETH만)")
                continue
            elif Temp_Check['currency'] == 'XRP' :
                print("# 보유자산 1개 = XRP만)")
                continue
            else :
                print("# 보유자산 Error")
                continue
        else :
            for x in res.json() :
                if x["currency"] == "KRW" :
                    My_Wallet_KRW = x
                elif x["currency"] == "BTC" :
                    My_Wallet_BIT = x # {'currency': 'BTC', 'balance': '0.00038666', 'locked': '0.0', 'avg_buy_price': '78524393.78', 'avg_buy_price_modified': False, 'unit_currency': 'KRW'}
                    # print("######## TESTTESTTESTSTEST")
                    # output.write("######## TESTTESTTESTSTEST\n")
                    # print(My_Wallet_BIT)
                elif x["currency"] == "ETH" :
                    My_Wallet_ETH = x # {'currency': 'ETH', 'balance': '0.00902136', 'locked': '0.0', 'avg_buy_price': '3141155.61', 'avg_buy_price_modified': False, 'unit_currency': 'KRW'}
                    # print("######## TESTTESTTESTSTEST")
                    # output.write("######## TESTTESTTESTSTEST\n")
                    # print(My_Wallet_ETH)
                elif x["currency"] == "XRP" :
                    My_Wallet_XRP = x # {'currency': 'XRP', 'balance': '4.57665903', 'locked': '0.0', 'avg_buy_price': '2185', 'avg_buy_price_modified': False, 'unit_currency': 'KRW'}
                    # print("######## TESTTESTTESTSTEST")
                    # output.write("######## TESTTESTTESTSTEST\n")
                    # print(My_Wallet_XRP)
                else :
                    print("# ELSE : My_Wallet_* Dictionary 초기화") # 보유 중인 COIN이 팔리면 초기화를 해줘야 한다. 안그러면 기존 정보로 LOOP가 시행되는 듯...???
                    My_Wallet_BIT = {}
                    My_Wallet_ETH = {}
                    My_Wallet_XRP = {}

        # print("# res.json() TYPE : %s" % type(res.json())) # <class 'list'>
        # print(res.json()) # LIST안에 Dictionary 형태
        Check_Currency = []
        for res_index in range(len(res.json())) :
            # print(type(res.json()[res_index])) # <class 'dict'>
            # print(res.json()[res_index]['currency'])
            Check_Currency.append(res.json()[res_index]['currency'])

        # print(df['코인'].to_list()) # <class 'pandas.core.series.Series'>
        Nothing_Currency = list(set(df['코인'].to_list()) ^ set(Check_Currency)) # set은 type이 set
        for Nothing_Buy in Nothing_Currency :
            if Nothing_Buy != 'KRW' :
                print("# %s : Nothing Buy ---> %s" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), Nothing_Buy))
                __Make_Put("BUY", Nothing_Buy, Call_Price_Possible, Min_Call_Price, OneDay_BUY_Count, File_Name)

        for x in res.json() :
            if x['currency'] != 'KRW' :

                # 매수 / 매도 조건
                # print(df.loc[df['코인'].str.match(str(x['currency'])),'목표수익'].values[0]) # <class 'numpy.float64'>
                Sell_Condition = df.loc[df['코인'].str.match(str(x['currency'])),'목표수익'].values[0]

                # 그냥 지정된 값으로 물타기
                # Buy_Condition = df.loc[df['코인'].str.match(str(x['currency'])),'물타기'].values[0]

                # 과거 평균값으로 물타기
                Buy_Condition = df.loc[df['코인'].str.match(str(x['currency'])),'Buy_Condition'].values[0]
                
                Market = "KRW-" + x['currency']
                price_KRW = Current_Coin(Market)
                Market_Coin_WON = round(float(x['balance']) * price_KRW, 1)
                My_Coin = upbit.get_balance(ticker=x['currency']) # <class 'float'>
                # Min_Call_Price = df.loc[df['코인'].str.match(str(x['currency'])),'1회매수액'].to_string()
                Min_Call_Price = df.loc[df['코인'].str.match(str(x['currency'])),'1회매수액'].values[0]
                Percent_My_Coin_Perc = round((price_KRW - float(x['avg_buy_price'])) / float(x['avg_buy_price']) * 100, 2)

                if Percent_My_Coin_Perc > Sell_Condition : # 조건만족 : 이익실현
                    print("# %s : 조건만족 (이익실현) : 수익조건 ---> %s, 물타기 조건 : %s, 현재는 %s, 내 %s는 %s (%s), 시장가격은 %s" % \
                        (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), Sell_Condition, Buy_Condition, Percent_My_Coin_Perc, Market, x['balance'], Market_Coin_WON, price_KRW))
                    output.write("# %s : 조건만족 (이익실현) : 수익조건 ---> %s, 물타기 조건 : %s, 현재는 %s, 내 %s는 %s (%s), 시장가격은 %s" % \
                        (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), Sell_Condition, Buy_Condition, Percent_My_Coin_Perc, Market, x['balance'], Market_Coin_WON, price_KRW))
                    __Make_Put("SELL", x['currency'], My_Coin, Min_Call_Price, OneDay_BUY_Count, File_Name)
                elif Percent_My_Coin_Perc < Buy_Condition : # 조건만족 : 물타기
                    print("# %s : 조건만족 (물타기) : 수익조건 ---> %s, 물타기 조건 : %s, 현재는 %s, 내 %s는 %s (%s), 시장가격은 %s" % \
                        (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), Sell_Condition, Buy_Condition, Percent_My_Coin_Perc, Market, x['balance'], Market_Coin_WON, price_KRW))
                    output.write("# %s : 조건만족 (물타기) : 수익조건 ---> %s, 물타기 조건 : %s, 현재는 %s, 내 %s는 %s (%s), 시장가격은 %s" % \
                        (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), Sell_Condition, Buy_Condition, Percent_My_Coin_Perc, Market, x['balance'], Market_Coin_WON, price_KRW))
                    Trade_Count = __Make_OrderLists(Market) # ---> Return 값 : bid : 매수, ask : 매도

                    # 아래 건수 만큼 매수가 이루어졌다면 매수 금지
                    Fixed_Trade_Count = 3
                    if Trade_Count > Fixed_Trade_Count :
                        print("# %s : %s 매수 %s건 이상 존재 (제한 %s건 이상)" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), x['currency'], Trade_Count, Fixed_Trade_Count))
                        output.write("# %s : %s 매수 %s건 이상 존재 (제한 %s건 이상)\n" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), x['currency'], Trade_Count, Fixed_Trade_Count))
                    else :
                        print("# %s : 매수 건 없음" % datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"))
                        output.write("# %s : 매수 건 없음\n" % datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"))
                        __Make_Put("BUY", x['currency'], My_Coin, Min_Call_Price, OneDay_BUY_Count, File_Name)
                        # CALL : 조건(BUY/SELL), COIN : 코인명, Call_Possible : 보유코인수, Min_Call_Price : 최소매수금액, 매수건수, 파일명

                else :
                    print("# %s : 조건에 안들어온다 WAIT! ---> 수익조건 : %s, 물타기 조건 : %s, 현재는 %s, 내 %s는 %s (%s), 시장가격은 %s" % \
                        (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), Sell_Condition, Buy_Condition, Percent_My_Coin_Perc, Market, x['balance'], Market_Coin_WON, price_KRW))
        print("")
                
        # print(My_Wallet_KRW) # {'currency': 'KRW', 'balance': '7087.59216207', 'locked': '0.0', 'avg_buy_price': '0', 'avg_buy_price_modified': True, 'unit_currency': 'KRW'}
        # print(My_Wallet_BIT) # {'currency': 'BTC', 'balance': '0.00013826', 'locked': '0.0', 'avg_buy_price': '73643511.93', 'avg_buy_price_modified': False, 'unit_currency': 'KRW'}
        # print(My_Wallet_ETH) # {'currency': 'ETH', 'balance': '0.00198886', 'locked': '0.0', 'avg_buy_price': '2514000', 'avg_buy_price_modified': False, 'unit_currency': 'KRW'}

        My_Wallet_Money = float(My_Wallet_KRW['balance']) # 내 보유 WON

        Now_Hour = datetime.datetime.today().strftime("%H")

        # print(Check_Hour)
        if Now_Hour not in Check_Hour :
            Check_Hour.append(Now_Hour)
            # print(Check_Hour)
            shutil.copyfile(File_Name, 'Bitpython.txt')
        else :
            print("No Update %s" % File_Name)

        output.close()
        time.sleep(15)

if __name__ == '__main__' :

    # start_time = "09:00"
    # stop_time = "15:30"

    OneDay_BUY_Count = 0
    # 하루에 매수하는 회수를 제어하기 위한 Count, PUT(BUY) 호출할 때마다 1 Count (Start_BIT_Trade에서 최초 0부터 해서 시작)

    DateTime = datetime.datetime.today().strftime("%Y%m%d")
    File_Name = "D:\\Python\\Log\\UPBit_History"+"_"+DateTime+".txt"
    output = open(File_Name, "a")
    output.write("# -*- coding: utf-8 -*-\n")

    ############################################################
    # Main Program
    # Temp_My_Wallet = Current_Query(OneDay_BUY_Count, File_Name)
    Call_Price_Possible, Call_Volume_Possible = Call_Condition(File_Name)
    Start_BIT_Trade(Call_Price_Possible, Call_Volume_Possible, File_Name)
    ############################################################

    ############################################################
    # print(upbit.get_balance("KRW-BTC")) # KRW-XRP 조회
    # print(upbit.get_balance("KRW")) # 보유 현금 조회

    # print("############################################################")
    # test1 = __Make_OrderLists("KRW-BTC")
    # print("# TEST1")
    # print(test1)
    # test2 = __Make_OrderLists("KRW-ETH")
    # print("# TEST2")
    # print(test2)
    # if test2 > 3 :
    #     print("Over")

       코인명   코인       목표수익        물타기    1회매수액     제한총액  Buy_Condition
0      KRW  KRW  1000000.0 -1000000.0  1000000  1000000            0.0
1  KRW-BTC  BTC        1.5       -1.5     6600    66000           -2.9
2  KRW-ETH  ETH        1.5       -1.5     6600    66000           -3.9
3  KRW-XRP  XRP        1.5       -1.5     6600    66000           -7.0
##########################
# BTC 마이너스만..
-0.12
-17.05
-5.890655737704917
# ETH 마이너스만..
-0.03
-29.67
-7.8484033613445385
# XRP 마이너스만..
-0.35
-54.52
-14.032427184466012
# WHILE - TEMP_BTC_Current : 70210000.0
# WHILE - TEMP_ETH_Current : 3030000.0
# WHILE - TEMP_XRP_Current : 1750.0
# 최소매수가능 금액 : 10000.000000
# 최소매수가능 BIT : 0.000142
# 최소매수가능 ETH : 0.003300
# 최소매수가능 XRP : 5.714286
# 1/10 Volume = 0.000104
# 1/10 Price = 430.187976

Min_Call_Price : 10000.0
Call_Price_Possbile : 430.18797562500004
# 1번째 Price : 430.18797562500004
# 2번째 Price : 860.3759512500001
# 3번째 Price : 1290.563926875
# 4번째 Price : 1720.7519025000001
# 5번째 Price : 2150.

# BID Count : 59
# ASK Count : 29
# Real BID Count #
# BID Count : 0
# 20210422 00:23:54 : 매수 건 없음
# 20210422 00:23:54 : __Make_Put Function Start
# 20210422 00:23:54 : BTC CALL Volume/Price : 0.001044개 or 원
# 20210422 00:23:54 :---> 매수해야되지만 보유금액이 안됨
# 20210422 00:23:55 : 조건만족 (물타기) : 수익조건 ---> 1.5, 물타기 조건 : -7.0, 현재는 -10.78, 내 KRW-XRP는 60.0151763 (105626.7), 시장가격은 1760.0
# BID Count : 20
# ASK Count : 7
# Real BID Count #
# BID Count : 0
# 20210422 00:23:55 : 매수 건 없음
# 20210422 00:23:55 : __Make_Put Function Start
# 20210422 00:23:55 : XRP CALL Volume/Price : 60.015176개 or 원
# 20210422 00:23:55 :---> 매수해야되지만 보유금액이 안됨
# 20210422 00:23:57 : 조건에 안들어온다 WAIT! ---> 수익조건 : 1.5, 물타기 조건 : -3.9, 현재는 -0.68, 내 KRW-ETH는 0.02756241 (83293.6), 시장가격은 3022000.0

No Update D:\Python\Log\UPBit_History_20210422.txt
# 20210422 00:24:12 : WHILE Start 
# 20210422 00:24:13 : 조건만족 (물타기) : 수익조건 ---> 1.5, 물타기 조건 : -2.9, 현재는 -5.49, 내 KRW-BTC는 0.00104369 (73093.8), 시장가격은 70034000.0
# BID Count : 59
# ASK Count : 

# 20210422 00:26:40 : 조건에 안들어온다 WAIT! ---> 수익조건 : 1.5, 물타기 조건 : -3.9, 현재는 -0.94, 내 KRW-ETH는 0.02756241 (83073.1), 시장가격은 3014000.0

No Update D:\Python\Log\UPBit_History_20210422.txt
# 20210422 00:26:55 : WHILE Start 
# 20210422 00:26:57 : 조건만족 (물타기) : 수익조건 ---> 1.5, 물타기 조건 : -2.9, 현재는 -5.53, 내 KRW-BTC는 0.00104369 (73059.3), 시장가격은 70001000.0
# BID Count : 59
# ASK Count : 29
# Real BID Count #
# BID Count : 0
# 20210422 00:26:58 : 매수 건 없음
# 20210422 00:26:58 : __Make_Put Function Start
# 20210422 00:26:58 : BTC CALL Volume/Price : 0.001044개 or 원
# 20210422 00:26:58 :---> 매수해야되지만 보유금액이 안됨
# 20210422 00:26:59 : 조건만족 (물타기) : 수익조건 ---> 1.5, 물타기 조건 : -7.0, 현재는 -10.78, 내 KRW-XRP는 60.0151763 (105626.7), 시장가격은 1760.0
# BID Count : 20
# ASK Count : 7
# Real BID Count #
# BID Count : 0
# 20210422 00:27:00 : 매수 건 없음
# 20210422 00:27:00 : __Make_Put Function Start
# 20210422 00:27:00 : XRP CALL Volume/Price : 60.015176개 or 원
# 20210422 00:27:00 :---> 매수해야되지만 보유금액이 안됨
# 20210422 00:27:01 : 조건에 안들어온다

# 20210422 00:29:42 : 조건만족 (물타기) : 수익조건 ---> 1.5, 물타기 조건 : -7.0, 현재는 -11.79, 내 KRW-XRP는 60.0151763 (104426.4), 시장가격은 1740.0
# BID Count : 20
# ASK Count : 7
# Real BID Count #
# BID Count : 0
# 20210422 00:29:43 : 매수 건 없음
# 20210422 00:29:43 : __Make_Put Function Start
# 20210422 00:29:43 : XRP CALL Volume/Price : 60.015176개 or 원
# 20210422 00:29:43 :---> 매수해야되지만 보유금액이 안됨
# 20210422 00:29:44 : 조건에 안들어온다 WAIT! ---> 수익조건 : 1.5, 물타기 조건 : -3.9, 현재는 -1.07, 내 KRW-ETH는 0.02756241 (82962.9), 시장가격은 3010000.0

No Update D:\Python\Log\UPBit_History_20210422.txt
# 20210422 00:29:59 : WHILE Start 
# 20210422 00:30:01 : 조건만족 (물타기) : 수익조건 ---> 1.5, 물타기 조건 : -2.9, 현재는 -5.81, 내 KRW-BTC는 0.00104369 (72841.2), 시장가격은 69792000.0
# BID Count : 59
# ASK Count : 29
# Real BID Count #
# BID Count : 0
# 20210422 00:30:02 : 매수 건 없음
# 20210422 00:30:02 : __Make_Put Function Start
# 20210422 00:30:02 : BTC CALL Volume/Price : 0.001044개 or 원
# 20210422 00:30:02 :---> 매수해야되지만 보유금액이 안됨
# 20210422 00:30:03 : 조건만족 (물타기

# 20210422 00:32:43 : WHILE Start 
# 20210422 00:32:44 : 조건만족 (물타기) : 수익조건 ---> 1.5, 물타기 조건 : -2.9, 현재는 -5.48, 내 KRW-BTC는 0.00104369 (73094.8), 시장가격은 70035000.0
# BID Count : 59
# ASK Count : 29
# Real BID Count #
# BID Count : 0
# 20210422 00:32:45 : 매수 건 없음
# 20210422 00:32:45 : __Make_Put Function Start
# 20210422 00:32:45 : BTC CALL Volume/Price : 0.001044개 or 원
# 20210422 00:32:45 :---> 매수해야되지만 보유금액이 안됨
# 20210422 00:32:46 : 조건만족 (물타기) : 수익조건 ---> 1.5, 물타기 조건 : -7.0, 현재는 -10.78, 내 KRW-XRP는 60.0151763 (105626.7), 시장가격은 1760.0
# BID Count : 20
# ASK Count : 7
# Real BID Count #
# BID Count : 0
# 20210422 00:32:47 : 매수 건 없음
# 20210422 00:32:47 : __Make_Put Function Start
# 20210422 00:32:47 : XRP CALL Volume/Price : 60.015176개 or 원
# 20210422 00:32:47 :---> 매수해야되지만 보유금액이 안됨
# 20210422 00:32:48 : 조건에 안들어온다 WAIT! ---> 수익조건 : 1.5, 물타기 조건 : -3.9, 현재는 -0.78, 내 KRW-ETH는 0.02756241 (83210.9), 시장가격은 3019000.0

No Update D:\Python\Log\UPBit_History_20210422.txt
# 20210422 00:33:03 : WHILE Sta

...

# 20210422 07:23:00 : 조건만족 (물타기) : 수익조건 ---> 1.5, 물타기 조건 : -2.9, 현재는 -7.03, 내 KRW-BTC는 0.00104369 (71900.8), 시장가격은 68891000.0
# BID Count : 59
# ASK Count : 29
# Real BID Count #
# BID Count : 0
# 20210422 07:23:01 : 매수 건 없음
# 20210422 07:23:01 : __Make_Put Function Start
# 20210422 07:23:01 : BTC CALL Volume/Price : 0.001044개 or 원
# 20210422 07:23:01 :---> 매수해야되지만 보유금액이 안됨
# 20210422 07:23:02 : 조건만족 (물타기) : 수익조건 ---> 1.5, 물타기 조건 : -7.0, 현재는 -14.32, 내 KRW-XRP는 60.0151763 (101425.6), 시장가격은 1690.0
# BID Count : 20
# ASK Count : 7
# Real BID Count #
# BID Count : 0
# 20210422 07:23:02 : 매수 건 없음
# 20210422 07:23:02 : __Make_Put Function Start
# 20210422 07:23:02 : XRP CALL Volume/Price : 60.015176개 or 원
# 20210422 07:23:03 :---> 매수해야되지만 보유금액이 안됨
# 20210422 07:23:04 : 조건에 안들어온다 WAIT! ---> 수익조건 : 1.5, 물타기 조건 : -3.9, 현재는 -1.2, 내 KRW-ETH는 0.02756241 (82852.6), 시장가격은 3006000.0

No Update D:\Python\Log\UPBit_History_20210422.txt
# 20210422 07:23:19 : WHILE Start 
# 20210422 07:23:20 : 조건만족 (물타기)

# BID Count : 20
# ASK Count : 7
# Real BID Count #
# BID Count : 0
# 20210422 07:25:43 : 매수 건 없음
# 20210422 07:25:43 : __Make_Put Function Start
# 20210422 07:25:43 : XRP CALL Volume/Price : 60.015176개 or 원
# 20210422 07:25:44 :---> 매수해야되지만 보유금액이 안됨
# 20210422 07:25:45 : 조건에 안들어온다 WAIT! ---> 수익조건 : 1.5, 물타기 조건 : -3.9, 현재는 -1.3, 내 KRW-ETH는 0.02756241 (82769.9), 시장가격은 3003000.0

No Update D:\Python\Log\UPBit_History_20210422.txt
# 20210422 07:26:00 : WHILE Start 
# 20210422 07:26:01 : 조건만족 (물타기) : 수익조건 ---> 1.5, 물타기 조건 : -2.9, 현재는 -7.18, 내 KRW-BTC는 0.00104369 (71785.0), 시장가격은 68780000.0
# BID Count : 59
# ASK Count : 29
# Real BID Count #
# BID Count : 0
# 20210422 07:26:02 : 매수 건 없음
# 20210422 07:26:02 : __Make_Put Function Start
# 20210422 07:26:02 : BTC CALL Volume/Price : 0.001044개 or 원
# 20210422 07:26:02 :---> 매수해야되지만 보유금액이 안됨
# 20210422 07:26:03 : 조건만족 (물타기) : 수익조건 ---> 1.5, 물타기 조건 : -7.0, 현재는 -14.58, 내 KRW-XRP는 60.0151763 (101125.6), 시장가격은 1685.0
# BID Count : 20
# ASK Count : 7

# 20210422 07:28:42 : 조건만족 (물타기) : 수익조건 ---> 1.5, 물타기 조건 : -2.9, 현재는 -7.35, 내 KRW-BTC는 0.00104369 (71649.3), 시장가격은 68650000.0
# BID Count : 59
# ASK Count : 29
# Real BID Count #
# BID Count : 0
# 20210422 07:28:43 : 매수 건 없음
# 20210422 07:28:43 : __Make_Put Function Start
# 20210422 07:28:43 : BTC CALL Volume/Price : 0.001044개 or 원
# 20210422 07:28:43 :---> 매수해야되지만 보유금액이 안됨
# 20210422 07:28:44 : 조건만족 (물타기) : 수익조건 ---> 1.5, 물타기 조건 : -7.0, 현재는 -15.34, 내 KRW-XRP는 60.0151763 (100225.3), 시장가격은 1670.0
# BID Count : 20
# ASK Count : 7
# Real BID Count #
# BID Count : 0
# 20210422 07:28:45 : 매수 건 없음
# 20210422 07:28:45 : __Make_Put Function Start
# 20210422 07:28:45 : XRP CALL Volume/Price : 60.015176개 or 원
# 20210422 07:28:45 :---> 매수해야되지만 보유금액이 안됨
# 20210422 07:28:46 : 조건에 안들어온다 WAIT! ---> 수익조건 : 1.5, 물타기 조건 : -3.9, 현재는 -1.6, 내 KRW-ETH는 0.02756241 (82521.9), 시장가격은 2994000.0

No Update D:\Python\Log\UPBit_History_20210422.txt
# 20210422 07:29:01 : WHILE Start 
# 20210422 07:29:03 : 조건만족 (물타기)

# BID Count : 20
# ASK Count : 7
# Real BID Count #
# BID Count : 0
# 20210422 07:31:26 : 매수 건 없음
# 20210422 07:31:26 : __Make_Put Function Start
# 20210422 07:31:26 : XRP CALL Volume/Price : 60.015176개 or 원
# 20210422 07:31:26 :---> 매수해야되지만 보유금액이 안됨
# 20210422 07:31:28 : 조건에 안들어온다 WAIT! ---> 수익조건 : 1.5, 물타기 조건 : -3.9, 현재는 -1.76, 내 KRW-ETH는 0.02756241 (82384.0), 시장가격은 2989000.0

No Update D:\Python\Log\UPBit_History_20210422.txt
# 20210422 07:31:43 : WHILE Start 
# 20210422 07:31:44 : 조건만족 (물타기) : 수익조건 ---> 1.5, 물타기 조건 : -2.9, 현재는 -7.59, 내 KRW-BTC는 0.00104369 (71469.8), 시장가격은 68478000.0
# BID Count : 59
# ASK Count : 29
# Real BID Count #
# BID Count : 0
# 20210422 07:31:45 : 매수 건 없음
# 20210422 07:31:45 : __Make_Put Function Start
# 20210422 07:31:45 : BTC CALL Volume/Price : 0.001044개 or 원
# 20210422 07:31:45 :---> 매수해야되지만 보유금액이 안됨
# 20210422 07:31:46 : 조건만족 (물타기) : 수익조건 ---> 1.5, 물타기 조건 : -7.0, 현재는 -15.59, 내 KRW-XRP는 60.0151763 (99925.3), 시장가격은 1665.0
# BID Count : 20
# ASK Count : 7

# 20210422 07:34:26 : 조건만족 (물타기) : 수익조건 ---> 1.5, 물타기 조건 : -2.9, 현재는 -7.62, 내 KRW-BTC는 0.00104369 (71445.8), 시장가격은 68455000.0
# BID Count : 59
# ASK Count : 29
# Real BID Count #
# BID Count : 0
# 20210422 07:34:26 : 매수 건 없음
# 20210422 07:34:26 : __Make_Put Function Start
# 20210422 07:34:26 : BTC CALL Volume/Price : 0.001044개 or 원
# 20210422 07:34:26 :---> 매수해야되지만 보유금액이 안됨
# 20210422 07:34:28 : 조건만족 (물타기) : 수익조건 ---> 1.5, 물타기 조건 : -7.0, 현재는 -15.34, 내 KRW-XRP는 60.0151763 (100225.3), 시장가격은 1670.0
# BID Count : 20
# ASK Count : 7
# Real BID Count #
# BID Count : 0
# 20210422 07:34:28 : 매수 건 없음
# 20210422 07:34:28 : __Make_Put Function Start
# 20210422 07:34:28 : XRP CALL Volume/Price : 60.015176개 or 원
# 20210422 07:34:28 :---> 매수해야되지만 보유금액이 안됨
# 20210422 07:34:29 : 조건에 안들어온다 WAIT! ---> 수익조건 : 1.5, 물타기 조건 : -3.9, 현재는 -2.06, 내 KRW-ETH는 0.02756241 (82136.0), 시장가격은 2980000.0

No Update D:\Python\Log\UPBit_History_20210422.txt
# 20210422 07:34:44 : WHILE Start 
# 20210422 07:34:46 : 조건만족 (물타기